# SQL
References:<br>
[https://www.khanacademy.org/computing/computer-programming/sql](https://www.khanacademy.org/computing/computer-programming/sql)<br>
[http://www.w3schools.com/sql/default.asp](http://www.w3schools.com/sql/default.asp)<br>
[https://www.sqlite.org/queryplanner.html](https://www.sqlite.org/queryplanner.html)
___

## Query planning for efficiency

### Using index tables to make data lookup more efficient.

Most tables in SQLite (or SQL) consist of zero or more rows with a unique integer key (the rowid or INTEGER PRIMARY KEY) followed by content. An example schema:
```
CREATE TABLE FruitsForSale(
  Fruit TEXT,
  State TEXT,
  Price REAL
);
```
<img src="images/tab.gif">

If the table is large (millions of rows or records), then it may take a long time to use rowid lookup to process a query such as,
```
SELECT price FROM fruitsforsale WHERE fruit='Peach';
```

To make this more efficient, an index table can be created,
```
CREATE INDEX Idx1 ON fruitsforsale(fruit);
```
<img src="images/idx1.gif">


Now when the query is executed, SQL will use ***binary search*** on the index to more rapidly find the appropriate item matching the `WHERE` clause.

Search time goes from an average case of $O(n/2)$ and worst case of $O(n)$ to an average and worst case of $O(\log_{2} n)$.

Two binary searches are necessary, the first to find the rowid for the appropriate item, then another on the full dataset to find the record with the correct rowid (since rowids are sorted in the original data table).

If ***multiple rows*** match the given query ('Orange' in the above example), then the number of binary searches needed increases with $k$ the number of matching rows: $~O[(k+1)\cdot \log_{2} n]$.

### Using index tables with multiple statements.

For `WHERE` clauses that contain ***multiple statements*** such as,
```
SELECT price FROM fruitsforsale WHERE fruit='Orange' AND state='CA';
```
Creating an additional index can further improve efficiency,
```
CREATE INDEX Idx2 ON fruitsforsale(state);
```
Efficiency improvements are only realized if the `ANALYZE` command has been run on the database so the SQL engine knows which index is more restrictive and will access that index first.

Additional improvement can be found by using a ***multi-column index***,
<img src="images/idx3.gif">

For the same query given above, only 2 binary searches are needed to find the appropriate values.

Finally, if the index includes the `price` column as well, then the original data table never needs accessing and the *number of binary searches needed can be cut in half*,
```
CREATE INDEX Idx4 ON FruitsForSale(fruit, state, price);
```
<img src="images/idx4lu1.gif">

These improvements while useful are still small compared to the large improvement of using binary search on an index rather than linear search on the entire data table.

For ***`OR`-connected statements***, multi-column indexing does not yield the same advantage. For example,
```
SELECT price FROM FruitsForSale WHERE fruit='Orange' OR state='CA';
```
can be executed with an `OR`-by-`UNION` process.
<img src="images/orquery.gif">

Here, only a single-column index for each column in the `OR` statement is needed.

### Using index tables to speed up sorting

Consider the query,
```
SELECT * FROM fruitsforsale ORDER BY fruit;
```
If a multi-column index is used that covers all columns of the table (`Idx4` above), then multiple rowid lookups can be prevented which will speed up the sorting process.

<img src="images/obfruitidx4.gif">

Here, the engine will simply traverse the index from one end to the other and return the appropriate row in time $O(n)$ and without having to allocate a large buffer to store the result set.

For ***searching and sorting*** at the same time,
```
SELECT * FROM fruitforsale ORDER BY fruit, price;
```
The engine does a single binary search for the range of rows in the covering index that satisfy the `WHERE` clause, the scans that range from top to bottom to get the desired results. The rows that satisfy the `WHERE` clause are guaranteed to be adjacent since the `WHERE` clause is an equality constraint on the left-most column of the index. And by scanning the matching index rows from top to bottom, the output is guaranteed to be ordered by `state` since the `state` column is the very next column to the right of the `fruit` column. And so the resulting query is very efficient.


In partial or ***block sorting***, only part of an `ORDER BY` clause can be satisfied using indexes. Consider, for example, the following query:
```
SELECT * FROM fruitforsale ORDER BY fruit, price;
```
If the covering index is used (`Idx4`), the first column will naturally be in the correct order but the third column (`price`) may be out of order. The engine will initiate a smaller sort to put these values in the correct order.

For this example then, there will be a single sort of 7 elements, then 1 sort of two elements.

<img src="images/partial-sort.gif">

The advantages of doing many smaller sorts instead of a single large sort are:
1. Multiple small sorts collectively use fewer CPU cycles than a single large sort.
1. Each small sort is run independently, meaning that much less information needs to be kept in temporary storage at any one time.
1. Those columns of the `ORDER BY` that are already in the correct order due to indexes can be omitted from the sort key, further reducing storage requirements and CPU time.
1. Output rows can be returned to the application as each small sort completes, and well before the table scan is complete.
1. If a `LIMIT` clause is present, it might be possible to avoid scanning the entire table.

___
## Query planning for safety

### Using transactions

When we issue a SQL command (such as `UPDATE`, `CREATE`, `INSERT`, or `DELETE`) that changes our database in some way, it starts what is called a ***transaction***. A transaction is a sequence of operations treated as a single logical piece of work (like a bank transaction), and in the world of databases, a transaction must comply to the [ACID](https://en.wikipedia.org/wiki/ACID) principles to make sure they are processed reliably.

The commands can be wrapped in a transaction and if one of the commands fail then the transaction will be rolled back and the database will be left as it was before it started.
```
BEGIN TRANSACTION;
UPDATE people SET husband = "Winston" WHERE user_id = 1;
UPDATE people SET wife = "Winnefer" WHERE user_id = 2;
COMMIT;
```
This also insures that commands issues from another user at the same time do not influence this transaction.

___
## Performance Tuning

### SQL EXPLAIN

Using `EXPLAIN` or `EXPLAIN PLAN` will provide information on the performance of an SQL query. Metrics such as `cost` define the performance. Available in most versions of SQL (Postgres, Oracle, MySQL, etc).